In [3]:
import requests
import psycopg2
import time
from datetime import datetime, timezone

# PostgreSQL 연결 설정 (postgres 데이터베이스로 연결)
conn = psycopg2.connect(
    host="localhost",     # PostgreSQL 서버 주소
    database="postgres",  # postgres 데이터베이스 이름
    user="postgres",      # PostgreSQL 사용자명
    password="0224"       # PostgreSQL 비밀번호
)
cursor = conn.cursor()

# 시가총액 상위 100개 코인 목록 가져오기
url = 'https://api.coingecko.com/api/v3/coins/markets'
params = {
    'vs_currency': 'usd',
    'order': 'market_cap_desc',
    'per_page': 100,  # 상위 100개 코인
    'page': 1  # 첫 페이지
}

response = requests.get(url, params=params)
print("Status Code:", response.status_code)
data = response.json()
time.sleep(1)  # 요청 후 대기 시간 조절 (1초)

# 각 코인의 데이터 처리
for coin in data:
    coin_id = coin['id']  # coin_id만 사용
    print(f"Fetching data for {coin_id}...")

    url = f'https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart'
    params = {
        'vs_currency': 'usd',
        'days': '7',  # 7일 간의 데이터
    }

    response = requests.get(url, params=params)
    while response.status_code == 429:
        print("Rate limit exceeded. Waiting for 1 minute...")
        time.sleep(60)  # 1분 대기 후 재요청
        response = requests.get(url, params=params)
    
    data = response.json()
    prices = data.get('prices', [])
    market_caps = data.get('market_caps', [])
    volumes = data.get('total_volumes', [])
    
    # 코인의 circulating_supply 및 total_supply 정보 가져오기
    url_supply = f'https://api.coingecko.com/api/v3/coins/{coin_id}'
    response_supply = requests.get(url_supply)
    circulating_supply = None
    total_supply = None
    if response_supply.status_code == 200:
        coin_details = response_supply.json()
        circulating_supply = coin_details.get('market_data', {}).get('circulating_supply')
        total_supply = coin_details.get('market_data', {}).get('total_supply')
    
    # 기본값 설정 (null 방지)
    circulating_supply = circulating_supply if circulating_supply is not None else 0
    total_supply = total_supply if total_supply is not None else 0
    
    # 각 일별 데이터를 PostgreSQL에 저장
    for price, market_cap, volume in zip(prices, market_caps, volumes):
        timestamp = datetime.fromtimestamp(price[0] / 1000, timezone.utc)
        price_value = price[1]
        market_cap_value = market_cap[1]
        volume_value = volume[1]

        cursor.execute("""
            INSERT INTO coin_statistics ("coin_id", "timestamp", "price", "market_cap", "volume", "circulating_supply", "total_supply")
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        """, (coin_id, timestamp, price_value, market_cap_value, volume_value, circulating_supply, total_supply))

    conn.commit()
    time.sleep(1)  # 1초 대기 후 다음 요청

# 연결 종료
cursor.close()
conn.close()
print("Data collection completed successfully!")

Status Code: 200
Fetching data for bitcoin...
Fetching data for ethereum...
Fetching data for tether...
Fetching data for ripple...
Rate limit exceeded. Waiting for 1 minute...
Fetching data for binancecoin...
Fetching data for solana...
Fetching data for usd-coin...
Fetching data for cardano...
Rate limit exceeded. Waiting for 1 minute...
Fetching data for dogecoin...
Fetching data for tron...
Fetching data for staked-ether...
Fetching data for pi-network...
Fetching data for wrapped-bitcoin...
Rate limit exceeded. Waiting for 1 minute...
Fetching data for leo-token...
Fetching data for chainlink...
Fetching data for hedera-hashgraph...
Rate limit exceeded. Waiting for 1 minute...
Fetching data for wrapped-steth...
Fetching data for usds...
Fetching data for stellar...
Rate limit exceeded. Waiting for 1 minute...
Fetching data for avalanche-2...
Fetching data for sui...
Fetching data for litecoin...
Fetching data for bitcoin-cash...
Rate limit exceeded. Waiting for 1 minute...
Fetchin